In [1]:
%pip install matplotlib torchinfo gdown -U

  Using cached matplotlib-3.10.3-cp312-cp312-win_amd64.whl.metadata (11 kB)
Using cached matplotlib-3.10.3-cp312-cp312-win_amd64.whl (8.1 MB)

  Attempting uninstall: matplotlib

    Found existing installation: matplotlib 3.10.1

    Uninstalling matplotlib-3.10.1:

   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
      Successfully uninstalled matplotlib-3.10.1
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ------------- -------------------------- 1/3 [matplotlib]
   ---------

# Pretrained Model

-   미리 학습된 모델.
-   **Pretrained model을 이용해 현재 문제를 해결한다.**
    -   새로 모델을 정의 하고 학습시키는 것 보다 Pretrained 모델을 사용해 모델을 정의하면 훨씬 좋은 성능의 모델을 만들 수 있다.
-   **Pretrain model을 사용하는 방식**
    -   제로샷 전이학습(Zero shot transfer learning)
        -   추가 학습없어 Pretrained 모델을 해결하려는 문제에 그대로 사용한다.
    -   전이학습 (Transfer learning)
        -   Pretrained 모델의 일부분을 재학습 시킨다. 주로 출력 Layer를 학습시킨다.
    -   미세조정 (Fine tuning)
        -   Pretrained 모델의 파라미터를 초기 파라미터로 사용하여 Custom Dataset으로 학습을 진행하여 모델의 모든 파라미터를 업데이트 시킨다.


## Pytorch에서 제공하는 Pretrained Model

-   분야별 라이브러리에서 제공
    -   torchvision: https://pytorch.org/vision/stable/models.html
-   torch hub 를 이용해 모델과 학습된 parameter를 사용할 수 있다.
    -   https://pytorch.org/hub/
-   이외에도 많은 모델과 학습된 paramter가 인터넷상에 공개되 있다.
    -   딥러닝 모델기반 application을 개발 할 때는 대부분 Transfer Learning을 한다.
    -   다양한 분야에서 연구된 많은 딥러닝 모델들이 구현되어 공개 되어 있으며 학습된 Parameter들도 제공되고 있다.
    -   [paperswithcode](https://paperswithcode.com/)에서 State Of The Art(SOTA) 논문들과 그 구현된 모델을 확인할 수 있다.

> **State Of The Art(SOTA)**: 특정 시점에 특정 분야에서 가장 성능이 좋은 모델을 말한다.


## VGGNet Pretrained 모델을 이용해 이미지 분류

-   Pytorch가 제공하는 VGG 모델은 ImageNet dataset으로 사전 학습 시킨 모델로 그 학습된 parameter를 제공한다.
    -   120만장의 trainset, 1000개의 class로 구성된 데이터셋.
    -   Output으로 1000개의 카테고리에 대한 확률을 출력한다.


In [2]:
# ImageNet 1000개의 class 목록
%pip install wget
import wget
url = 'https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt'
imagenet_filepath = wget.download(url) # url의 파일을 다운로드.

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9713 sha256=ba5651d63cbce00d024ff7e269644b6090d7e3a2939781cba0a8e09480dd74cf
  Stored in directory: c:\users\playdata\appdata\local\pip\cache\wheels\01\46\3b\e29ffbe4ebe614ff224bad40fc6a5773a67a163251585a13a9
Successfully built wget
Note: you may need to restart the kernel to use updated packages.


  DEPRECATION: Building 'wget' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wget'. Discussion can be found at https://github.com/pypa/pip/issues/6334


In [3]:
import ast
with open("imagenet1000_clsidx_to_labels.txt", "rt") as fr:
    index_to_class = ast.literal_eval(fr.read())  # dictionary로 변환.
print(type(index_to_class), len(index_to_class))

<class 'dict'> 1000


In [4]:
for key, value in list(index_to_class.items())[:5]:
    print(key, value)

0 tench, Tinca tinca
1 goldfish, Carassius auratus
2 great white shark, white shark, man-eater, man-eating shark, Carcharodon carcharias
3 tiger shark, Galeocerdo cuvieri
4 hammerhead, hammerhead shark


### Pretrained Model Load 및 추론

In [ ]:
import torch
from torchvision import models, transforms  # torchvision.models: Pretrained 모델들을 제공.
from torchinfo import summary

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)


In [ ]:
## Pretrained 모델 Loading
load_model = models.vgg16(
    weights=models.VGG16_Weights.DEFAULT # 학습된 weight(parameter) 도 같이 load
)

In [ ]:
# 추론할 이미지 다운로드
import requests
from io import BytesIO
from PIL import Image

img_url = 'https://cdn.download.ams.birds.cornell.edu/api/v1/asset/169231441/1800'
img_url = 'https://blogs.ifas.ufl.edu/news/files/2021/10/anole-FB.jpg'
img_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/YellowLabradorLooking_new.jpg/640px-YellowLabradorLooking_new.jpg'

res = requests.get(img_url)
# res.content # binary data
# BytesIO() -> binary data(file)을 bytes 타입으로 변환.

test_img = Image.open(BytesIO(res.content)) # res: http 응답정보. res.content: 다운받은 binary 파일
test_img

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor()
])
input_tensor = transform(test_img).unsqueeze(0)
input_tensor.shape

In [ ]:
load_model = load_model.to(device)
input_tensor = input_tensor.to(device)

load_model.eval()
with torch.no_grad():
    pred = load_model(input_tensor)

In [ ]:
pred.shape

In [ ]:
pred = torch.nn.Softmax(dim=-1)(pred)
pred_cls = pred.max(dim=-1).indices[0]
pred_proba = pred.max(dim=-1).values[0]

In [ ]:
print(pred_proba)
print(pred_cls, index_to_class[pred_cls.item()])

# Transfer learning (전이학습)

-   사전에 학습된 신경망의 구조와 파라미터를 재사용해서 새로운 모델(우리가 만드는 모델)의 시작점으로 삼고 해결하려는 문제를 위해 다시 학습시킨다.
-   전이 학습을 통해 다음을 해결할 수 있다.
    1. 데이터 부족문제
        - 딥러닝은 대용량의 학습데이터가 필요하다.
        - 충분한 데이터를 수집하는 것은 항상 어렵다.
    2. 과다한 계산량
        - 신경망 학습에는 엄청난 양의 계산 자원이 필요하다.


![transfer_learning01](figures/09_transfer_01.png)


-   미리 학습된(pre-trained) Model을 이용하여 모델을 구성한 뒤 현재 하려는 예측 문제를 해결한다.
-   보통 Pretrained Model에서 Feature Extraction 부분을 사용한다.
    -   Computer Vision 문제의 경우 Bottom 쪽의 Convolution Layer(Feature Extractor)들은 이미지에 나타나는 일반적인 특성을 추출하므로 **다른 대상을 가지고 학습했다고 하더라도 재사용할 수 있다.**
    -   Top 부분 Layer 부분은 특히 출력 Layer의 경우 대상 데이터셋의 목적에 맞게 변경 해야 하므로 재사용할 수 없다.


![transfer_learning02](figures/09_transfer_02.png)

> **Frozon**: Training시 parameter가 update 되지 않도록 하는 것을 말한다.


### Feature extraction 재사용

-   Pretrained Model에서 Feature Extractor 만 가져오고 추론기(Fully connected layer)만 새로 정의한 뒤 그 둘을 합쳐서 모델을 만든다.
-   학습시 직접 구성한 추론기만 학습되도록 한다.
    -   Feature Extractor는 추론을 위한 Feature 추출을 하는 역할만 하고 그 parameter(weight)가 학습되지 않도록 한다.
-   모델/레이어의 parameter trainable 여부 속성 변경
    -   model/layer 의 `parameters()` 메소드를 이용해 weight와 bias를 조회한 뒤 `requires_grad` 속성을 `False`로 변경한다.

#### Backbone, Base network

-   전체 네트워크에서 Feature Extraction의 역할을 담당하는 부분을 Backbone/Base network라고 한다.
-   추론을 담당하는 layer block은 Head Network 이라고 한다.


## Fine-tuning(미세조정)

-   Transfer Learning을 위한 Pretrained 모델을 내가 학습시켜야 하는 데이터셋(Custom Dataset)으로 **재학습**시키는 것을 fine tuning 이라고 한다.
-   주어진 문제에 더 적합하도록 Feature Extractor의 가중치들도 조정 한다.


### Fine tuning 전략

![transfer02](figures/09_transfer_03.png)

-   **세 전략 모두 추론기는 trainable로 한다.**


**<span style='font-size:1.2em'>1. 전체 모델을 전부 학습시킨다.(1번)</span>**

-   Pretrained 모델의 weight는 Feature extraction 의 초기 weight 역할을 한다.
-   **Train dataset의 양이 많고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **낮은 경우** 적용.
-   학습에 시간이 많이 걸린다.

**<span style='font-size:1.2em'>2. Pretrained 모델 Bottom layer들(Input과 가까운 Layer들)은 고정시키고 Top layer의 일부를 재학습시킨다.<span>**

-   **Train dataset의 양이 많고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **높은 경우** 적용.
-   **Train dataset의 양이 적고** Pretained 모델이 학습했던 dataset과 custom dataset의 class간의 유사성이 **낮은 경우** 적용

**<span style='font-size:1.2em'>3. Pretrained 모델 전체를 고정시키고 classifier layer들만 학습시ont3번)</span>**

-   **Train dataset의 양이 적고** Pretrained 모델이 학습했던 dataset과 Custom dataset의 class간의 유사성이 **높은 경우** 적용.

> **Custom dataset:** 내가 학습시키고자 하는 dataset
> 1번 2번 전략을 Fine tuning 이라고 한다.

![fine tuning](figures/09_finetuning.png)


In [ ]:
%pip install gdown -U

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim

from torchvision import models, datasets, transforms

from torchinfo import summary

from module.train import fit, test_multi_classification
from module.utils import plot_fit_result

import os
from zipfile import ZipFile


import gdown

device = 'cuda' if torch.cuda.is_available() else "cpu"
device

In [ ]:
os.makedirs("data", exist_ok=True)

In [ ]:
# download
url = "https://drive.google.com/uc?id=1YIxDL0XJhhAMdScdRUfDgccAqyCw5-ZV"
path = r"data/cats_and_dogs_small.zip" # 다운 받은 공유파일을 저장할 경로.
gdown.download(url, path, quiet=False)

In [ ]:
# 압축 풀기
os.makedirs("data", exist_ok=True)
target_path = 'data/cats_and_dogs'
with ZipFile(path) as zf:
    zf.extractall(target_path)

## Dataset, DataLoader

In [ ]:
target_path = 'data/cats_and_dogs'
BATCH_SIZE = 64
EPOCH = 10
LEARNING_RATE = 0.001

In [ ]:
# Image Augmentation+전처리 설정
train_transform = transforms.Compose([
    transforms.Resize((224, 224)), # resize
    transforms.ToTensor(),         # ndarray, Image -> Tensor, 0 ~ 1 정규화, channel first
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    # 채널별 평균, 표준편차설정. -> Standard Scaling 처리.(픽셀값-평균)/표준편차
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(), 
    transforms.RandomRotation(degrees=180)
])
test_transform = transforms.Compose([
    transforms.Resize((224, 224)), #  resize
    transforms.ToTensor(),          # ndarray, Image -> Tensor, 0 ~ 1 정규화, channel first
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [ ]:
# Dataset 정의
train_set = datasets.ImageFolder(
    os.path.join(target_path, "train"), # Data들을 저장한 디렉토리.
    transform=train_transform
)
valid_set = datasets.ImageFolder(
    os.path.join(target_path, "validation"),
    transform=test_transform
)
test_set = datasets.ImageFolder(
    os.path.join(target_path, "test"),
    transform=test_transform
)

In [ ]:
# class, class_name 확인
print(train_set.classes)
print(train_set.class_to_idx)

In [ ]:
# DataLoader
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, drop_last=True,  
                          num_workers=os.cpu_count()) # 데이터 불러오는 것 병렬처리.
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, num_workers=os.cpu_count())
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, num_workers=os.cpu_count())

In [ ]:
# step 수
len(train_loader), len(test_loader), len(valid_loader)

In [ ]:
### Trainset Image 확인
import matplotlib.pyplot as plt
img = train_set[0][0]
img = img.permute(1, 2, 0)
img.shape
print(train_set[0][1])
plt.imshow(img)
plt.show()

## 모델 정의, 학습

In [ ]:
# Transfer Learning - Backbone 모델: VGG16 + classifier(내것)
model = models.vgg16(models.VGG16_Weights.DEFAULT)
model

In [ ]:
# Frozon -> 파라미터들이 학습시 update 되지 않도록 변경.
for p in model.parameters():
    p.requires_grad = False

In [ ]:
# 분류기(classifier)를 내것으로 변경.
model.classifier = nn.Linear(in_features=25088, out_features=2)

In [ ]:
summary(model, (1, 3, 224, 224))

## 학습

In [ ]:
os.makedirs("saved_models", exist_ok=True)
save_model_path = "saved_models/cat_dog_model.pt"

model = model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

result = fit(train_loader, valid_loader, model, loss_fn, optimizer, EPOCH,
            save_best_model=True, save_model_path=save_model_path, 
            device=device, mode="multi")

## 최종평가

In [ ]:
load_model = torch.load(save_model_path)

In [ ]:
loss, acc = test_multi_classification(
    test_loader, load_model, loss_fn, device=device
)
loss, acc

In [ ]:
import torch 
from PIL import Image

def predict(image_path, model, transform, device):
    # "model로 image_path의 이미지를 추론한 결과를 반환."
    img = Image.open(image_path)  # 추론대상 이미지 loading
    input_data = transform(img)  # shape: (C, H, W)
    input_data = input_data.unsqueeze(dim=0) # (C, H, W) -> (1, C, H, W) 
    input_data = input_data.to(device)

    # 추론
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        pred = model(input_data)
        pred_proba = pred.softmax(dim=-1) # 확률값으로 변경.
        pred_label = pred_proba.argmax(dim=-1).item()  # Tensor([3]) -> 3
        pred_proba_max = pred_proba.max(dim=-1).values.item()
        class_name = "cat" if pred_label == 0 else "dog"
        return pred_label, class_name, pred_proba_max

In [ ]:
predict("test_img/dog.jpg", load_model, test_transform, device)

In [ ]:
predict("test_img/cat.jpg", load_model, test_transform, device)

In [ ]:
predict("test_img/image.jpg", load_model, test_transform, device)